In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import chi2_contingency, norm
from data_viz import *

In [2]:
data = pd.read_csv("data/animal_center.csv")
data

,animal_id,name,animal_type,sex_intake,sex_outcome,breed,color,date_of_birth,found_location,intake_datetime,outcome_datetime,intake_type,intake_condition,outcome_type,outcome_subtype,age_upon_intake(years),age_upon_outcome(years),duration(days)
0,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,8700 Research in Austin (TX),2014-03-07 14:26:00,2014-03-08 17:10:00,Public Assist,Normal,Return to Owner,NaN,6.7,6.7,1.0
1,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,8700 Research Blvd in Austin (TX),2014-12-19 10:21:00,2014-12-20 16:35:00,Public Assist,Normal,Return to Owner,NaN,7.4,7.4,1.0
2,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,Colony Creek And Hunters Trace in Austin (TX),2017-12-07 14:07:00,2017-12-07 00:00:00,Stray,Normal,Return to Owner,NaN,10.4,10.4,-1.0
3,A047759,Oreo,Dog,Neutered Male,Neutered Male,Dachshund,Tricolor,2004-04-02,Austin (TX),2014-04-02 15:55:00,2014-04-07 15:12:00,Owner Surrender,Normal,Transfer,Partner,10.0,10.0,4.0
4,A134067,Bandit,Dog,Neutered Male,Neutered Male,Shetland Sheepdog,Brown/White,1997-10-16,12034 Research Blvd in Austin (TX),2013-11-16 09:02:00,2013-11-16 11:54:00,Public Assist,Injured,Return to Owner,NaN,16.1,16.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157585,A894036,NaN,Dog,Intact Female,NaN,Miniature Pinscher Mix,Black/Tan,NaN,Martin Luther King Jr Blvd And M Sta Ln in Aus...,2023-12-01 17:33:00,NaN,Stray,Normal,NaN,NaN,NaN,NaN,NaN
157586,A894054,Dawn,Dog,Intact Female,Intact Female,Chihuahua Shorthair,Tan,2022-12-02,2001 Ponciana Loop in Austin (TX),2023-12-02 09:52:00,2023-12-02 13:03:00,Stray,Normal,Rto-Adopt,NaN,1.0,1.0,0.0
157587,A894063,Logan,Cat,Neutered Male,NaN,Domestic Shorthair,Brown Tabby/White,NaN,8585 Scotland Well Drive in Austin (TX),2023-12-02 12:49:00,NaN,Stray,Normal,NaN,NaN,NaN,NaN,NaN
157588,A894067,Maverick,Dog,Intact Male,NaN,Siberian Husky,Black/White,NaN,124 West Anderson Lane in Austin (TX),2023-12-02 13:53:00,NaN,Abandoned,Normal,NaN,NaN,NaN,NaN,NaN


## **Hypothesis3:**
Does intake condition influence the probability of adoption for animals?

$H0$: There is no significant relationship between the intake condition of animals at the shelter and their likelihood of adoption.\
$H1$: There is a significant relationship between the intake condition of animals at the shelter and their likelihood of adoption.

In [3]:
abnormal_cond = ["Injured", "Sick", "Nursing", "Neonatal", "Aged", "Medical",
                 "Feral", "Pregnant", "Behavior", "Med Attn", "Med Urgent",
                 "Neurologic", "Space", "Parvo", "Agonal", "Panleuk", "Congenital"]

In [4]:
data.columns

Index(['animal_id', 'name', 'animal_type', 'sex_intake', 'sex_outcome',
       'breed', 'color', 'date_of_birth', 'found_location', 'intake_datetime',
       'outcome_datetime', 'intake_type', 'intake_condition', 'outcome_type',
       'outcome_subtype', 'age_upon_intake(years)', 'age_upon_outcome(years)',
       'duration(days)'],
      dtype='object')

In [5]:
condition_research = data[~data["intake_condition"].isin(["Other", "Unknown"])]\
                    [["animal_id", "intake_condition", "outcome_type"]].reset_index(drop=True)
condition_research

,animal_id,intake_condition,outcome_type
0,A006100,Normal,Return to Owner
1,A006100,Normal,Return to Owner
2,A006100,Normal,Return to Owner
3,A047759,Normal,Transfer
4,A134067,Injured,Return to Owner
...,...,...,...
157229,A894036,Normal,NaN
157230,A894054,Normal,Rto-Adopt
157231,A894063,Normal,NaN
157232,A894067,Normal,NaN


In [6]:
condition_research["intake_cond_binary"] = np.where(condition_research["intake_condition"].isin(abnormal_cond), "Non-Normal", "Normal")
condition_research["adoption_status"] = np.where(condition_research["outcome_type"]=="Adoption", "Adopted", "Not Adopted")
condition_research

,animal_id,intake_condition,outcome_type,intake_cond_binary,adoption_status
0,A006100,Normal,Return to Owner,Normal,Not Adopted
1,A006100,Normal,Return to Owner,Normal,Not Adopted
2,A006100,Normal,Return to Owner,Normal,Not Adopted
3,A047759,Normal,Transfer,Normal,Not Adopted
4,A134067,Injured,Return to Owner,Non-Normal,Not Adopted
...,...,...,...,...,...
157229,A894036,Normal,NaN,Normal,Not Adopted
157230,A894054,Normal,Rto-Adopt,Normal,Not Adopted
157231,A894063,Normal,NaN,Normal,Not Adopted
157232,A894067,Normal,NaN,Normal,Not Adopted


In [7]:
condition_adoption_contingency_table = pd.crosstab(condition_research["intake_cond_binary"], condition_research["adoption_status"])
condition_adoption_contingency_table

adoption_status,Adopted,Not Adopted
intake_cond_binary,,
Non-Normal,5957,16627
Normal,67752,66898


In [8]:
condition_list = ["Non-Normal", "Normal"]

for condition in condition_list:
    total = condition_adoption_contingency_table.loc[condition].sum()
    adoption_cnt = condition_adoption_contingency_table.loc[condition, "Adopted"]
    adoption_rate = round(adoption_cnt/total, 4)*100.0
    print(f"{adoption_rate}% of animal whose condition is {condition.lower()} have been adopted.")

26.38% of animal whose condition is non-normal have been adopted.
50.32% of animal whose condition is normal have been adopted.


In [9]:
chi2, p, dof, expected = chi2_contingency(condition_adoption_contingency_table)
print(f"chi2:    {chi2}")
print(f"p-value: {p}")
print(f"dof:     {dof}")
print(f"expected:\n{expected}")

chi2:    4450.128225289967
p-value: 0.0
dof:     1
expected:
[[10587.04895888 11996.95104112]
 [63121.95104112 71528.04895888]]


In [10]:
# set significance level = 0.05
alpha = 0.05
if p < alpha:
    print("There is a significant relationship between the intake condition of animals at the shelter and their likelihood of adoption.")
else:
    print("There is no significant relationship between the intake condition of animals at the shelter and their likelihood of adoption.")

There is a significant relationship between the intake condition of animals at the shelter and their likelihood of adoption.


In [11]:
fig = px.histogram(condition_research,
             x="intake_cond_binary",
             color="adoption_status",
             barmode = "group",
             title="Adoption Status by Intake Condition")
# fig.update_traces(name="")
fig.update_layout(title_x=0.5,
                  xaxis=dict(title="Adoption Status"),
                  yaxis=dict(title="Counts"),
                  legend_title_text="Adoption Status",
                  height=600,
                  width=1000)


fig.show()